### Install and Import Dependencies

In [ ]:
!pip install tsfresh

In [ ]:
import pandas as pd
import numpy as np
import tsfresh
from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import shuffle
import tensorflow as tf
import os
import itertools
import matplotlib.pyplot as plt

# keras goodies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPooling1D, BatchNormalization, LSTM, SimpleRNN
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics as kmetrics
import tensorflow.keras.backend as K

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Respeck_recordings_clean.csv to Respeck_recordings_clean.csv


### Data Cleaning

In [ ]:
base_df = pd.read_csv('Respeck_recordings_clean.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
len(base_df)

492676

In [ ]:
columns_of_interest = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

In [ ]:
nan_indices = set()
for i in range(len(base_df)):
    for col in columns_of_interest:
        if pd.isnull(base_df.loc[i, col]):
            nan_indices.add(i)

In [ ]:
nan_indices

{340128}

In [ ]:
base_df.drop(list(nan_indices), inplace=True)

In [ ]:
base_df

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,sensor_type,activity_type,activity_code,subject_id,notes,recording_id
0,1.633516e+12,0.261475,-1.116516,-0.502991,-0.812500,12.312500,19.500000,Respeck,Climbing stairs,12.0,s1870467,rob,Respeck_s1870467_Climbing stairs_06-10-2021_11...
1,1.633516e+12,-0.177734,-0.636292,-0.477600,-5.265625,2.953125,-9.281250,Respeck,Climbing stairs,12.0,s1870467,rob,Respeck_s1870467_Climbing stairs_06-10-2021_11...
2,1.633516e+12,0.351562,-0.879456,-0.335754,5.671875,24.656250,-10.562500,Respeck,Climbing stairs,12.0,s1870467,rob,Respeck_s1870467_Climbing stairs_06-10-2021_11...
3,1.633516e+12,0.298584,-1.253479,-0.299622,-3.609375,2.687500,5.890625,Respeck,Climbing stairs,12.0,s1870467,rob,Respeck_s1870467_Climbing stairs_06-10-2021_11...
4,1.633516e+12,0.056152,-0.777405,-0.210754,-14.328125,5.421875,8.140625,Respeck,Climbing stairs,12.0,s1870467,rob,Respeck_s1870467_Climbing stairs_06-10-2021_11...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492671,1.632930e+12,-0.376465,-0.746399,0.693543,-0.171875,-0.953125,1.281250,Respeck,Sitting bent backward,5.0,s2211228,NaN,Respeck_s2211228_Sitting bent backward_29-09-2...
492672,1.632930e+12,-0.345703,-0.749329,0.677429,-0.671875,-0.187500,-1.015625,Respeck,Sitting bent backward,5.0,s2211228,NaN,Respeck_s2211228_Sitting bent backward_29-09-2...
492673,1.632930e+12,-0.368408,-0.745422,0.690857,-0.218750,-0.640625,0.375000,Respeck,Sitting bent backward,5.0,s2211228,NaN,Respeck_s2211228_Sitting bent backward_29-09-2...
492674,1.632930e+12,-0.363281,-0.747375,0.689148,0.078125,0.796875,-0.046875,Respeck,Sitting bent backward,5.0,s2211228,NaN,Respeck_s2211228_Sitting bent backward_29-09-2...


### Data Processing

In [ ]:
window_size = 50 # 50 datapoints for the window size, which, at 25Hz, means 2 seconds
step_size = 25 # this is 50% overlap

window_number = 0 # start a counter at 0 to keep track of the window number

all_overlapping_windows = []

for rid, group in base_df.groupby("recording_id"):
    print(f"Processing rid = {rid}")
    
    large_enough_windows = [window for window in group.rolling(window=window_size, min_periods=window_size) if len(window) == window_size]
    
    if len(large_enough_windows) == 0:
      continue

    overlapping_windows = large_enough_windows[::step_size] 
    
    # then we will append a window ID to each window
    for window in overlapping_windows:
        window.loc[:, 'window_id'] = window_number
        window_number += 1
    
    
    all_overlapping_windows.append(pd.concat(overlapping_windows).reset_index(drop=True))

Processing rid = Respeck_s1541031_Climbing stairs_06-10-2021_11-25-43
Processing rid = Respeck_s1541031_Descending stairs_06-10-2021_11-25-05


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Processing rid = Respeck_s1541031_Desk work_06-10-2021_11-04-49
Processing rid = Respeck_s1541031_Lying down left_06-10-2021_14-30-04
Processing rid = Respeck_s1541031_Lying down on back_06-10-2021_11-15-13
Processing rid = Respeck_s1541031_Lying down on stomach_06-10-2021_14-30-50
Processing rid = Respeck_s1541031_Lying down right_06-10-2021_14-27-40
Processing rid = Respeck_s1541031_Movement_06-10-2021_11-11-18
Processing rid = Respeck_s1541031_Running_08-10-2021_12-10-37
Processing rid = Respeck_s1541031_Sitting bent backward_06-10-2021_14-33-45
Processing rid = Respeck_s1541031_Sitting bent forward_06-10-2021_14-32-55
Processing rid = Respeck_s1541031_Sitting_06-10-2021_10-25-01
Processing rid = Respeck_s1541031_Standing_06-10-2021_11-10-10
Processing rid = Respeck_s1541031_Walking at normal speed_06-10-2021_18-30-14
Processing rid = Respeck_s1704145_Climbing stairs_23-09-2021_11-34-34
Processing rid = Respeck_s1704145_Descending stairs_23-09-2021_11-30-25
Processing rid = Respeck_

In [ ]:
np.unique(base_df['activity_type'])

array(['Climbing stairs', 'Descending stairs', 'Desk work',
       'Falling on knees', 'Falling on the back', 'Falling on the left',
       'Falling on the right', 'Lying down left', 'Lying down on back',
       'Lying down on stomach', 'Lying down right', 'Movement', 'Running',
       'Sitting', 'Sitting bent backward', 'Sitting bent forward',
       'Standing', 'Walking at normal speed'], dtype=object)

In [25]:
final_sliding_windows = pd.concat(all_overlapping_windows).reset_index(drop=True)

class_labels = {
    'Climbing stairs': 3,
    'Descending stairs': 3,
    'Desk work': 1,
    'Falling on knees': 0,
    'Falling on the back': 0,
    'Falling on the left': 0,
    'Falling on the right': 0,
    'Lying down left': 2,
    'Lying down on back': 2,
    'Lying down on stomach': 2,
    'Lying down right': 2,
    'Movement': 4,
    'Running': 4,
    'Sitting': 1,
    'Sitting bent backward': 1,
    'Sitting bent forward': 1,
    'Standing': 1,
    'Walking at normal speed': 3 
}

X = []
y = []

for window_id, group in final_sliding_windows.groupby('window_id'):
    #print(f"window_id = {window_id}")
    
    shape = group[columns_of_interest].values.shape
    #print(f"shape = {shape}")
    
    X.append(group[columns_of_interest].values)
    y.append(class_labels[group["activity_type"].values[0]])

    if class_labels[group["activity_type"].values[0]] == 0:
      #print("Oversampling...")
      for _ in range(80):
        X.append(group[columns_of_interest].values)
        y.append(class_labels[group["activity_type"].values[0]])

X = np.asarray(X)
y = np.asarray(y)
X, y = shuffle(X, y)

print(f"X shape = {X.shape}")
print(f"y shape = {y.shape}")

X shape = (23807, 50, 6)
y shape = (23807,)


In [26]:
for i in range(5):
  print(i, ": ", len(y[y==i]))

0 :  5427
1 :  6712
2 :  5412
3 :  3588
4 :  2668


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, train_size=0.8)

y_train = np.asarray(pd.get_dummies(y_train), dtype=np.float32)
y_test = np.asarray(pd.get_dummies(y_test), dtype=np.float32)

In [28]:
print(f"X_train shape = {X_train.shape}")
print(f"y_train shape = {y_train.shape}")

print(f"X_test shape = {X_test.shape}")
print(f"y_test shape = {y_test.shape}")

X_train shape = (19045, 50, 6)
y_train shape = (19045, 5)
X_test shape = (4762, 50, 6)
y_test shape = (4762, 5)


### Model Training (CNN)

In [29]:
filters = 64
kernel_size = 3
n_features = 6
activation='relu'
n_classes = 5

model = Sequential()

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear', 
                 input_shape=(window_size, n_features)))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 48, 64)            1216      
_________________________________________________________________
batch_normalization_3 (Batch (None, 48, 64)            256       
_________________________________________________________________
activation_3 (Activation)    (None, 48, 64)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 46, 64)            12352     
_________________________________________________________________
batch_normalization_4 (Batch (None, 46, 64)            256       
_________________________________________________________________
activation_4 (Activation)    (None, 46, 64)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 44, 64)           

In [30]:
model.compile(
    optimizer=optimizers.SGD(lr=0.001),
    loss='binary_crossentropy',
    metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [31]:
model.fit(X_train, y_train,
        batch_size=10, epochs=50)

Epoch 1/50
1905/1905 [==============================] - 23s 12ms/step - loss: 0.3585 - accuracy: 0.5569
Epoch 2/50
1905/1905 [==============================] - 21s 11ms/step - loss: 0.2737 - accuracy: 0.6438
Epoch 3/50
1905/1905 [==============================] - 22s 11ms/step - loss: 0.2526 - accuracy: 0.6725
Epoch 4/50
1905/1905 [==============================] - 21s 11ms/step - loss: 0.2410 - accuracy: 0.6843
Epoch 5/50
1905/1905 [==============================] - 21s 11ms/step - loss: 0.2336 - accuracy: 0.6916
Epoch 6/50
1905/1905 [==============================] - 21s 11ms/step - loss: 0.2267 - accuracy: 0.7116
Epoch 7/50
1905/1905 [==============================] - 21s 11ms/step - loss: 0.2180 - accuracy: 0.7488
Epoch 8/50
1905/1905 [==============================] - 20s 11ms/step - loss: 0.1989 - accuracy: 0.8071
Epoch 9/50
1905/1905 [==============================] - 20s 11ms/step - loss: 0.1600 - accuracy: 0.8608
Epoch 10/50
1905/1905 [==============================] - 21s 11m

In [32]:
# stats for five classes
y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
print("*" * 80)
print("Classification report")
print("*" * 80)
print(classification_report(y_true_labels, y_pred_labels))

********************************************************************************
Classification report
********************************************************************************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1117
           1       0.94      0.89      0.92      1281
           2       0.88      0.95      0.92      1108
           3       0.86      0.91      0.89       732
           4       0.89      0.77      0.82       524

    accuracy                           0.92      4762
   macro avg       0.91      0.90      0.91      4762
weighted avg       0.92      0.92      0.92      4762



In [33]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model_cnn.tflite', 'wb') as f:
  f.write(tflite_model)

from google.colab import files
files.download('model_cnn.tflite')

INFO:tensorflow:Assets written to: /tmp/tmpssjyouyj/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.fit(X_train, y_train,
        batch_size=10, epochs=50)

Epoch 1/50
1905/1905 [==============================] - 25s 12ms/step - loss: 0.3503 - accuracy: 0.6658
Epoch 2/50
1905/1905 [==============================] - 23s 12ms/step - loss: 0.2483 - accuracy: 0.7479
Epoch 3/50
1905/1905 [==============================] - 23s 12ms/step - loss: 0.2215 - accuracy: 0.7690
Epoch 4/50
1905/1905 [==============================] - 23s 12ms/step - loss: 0.2061 - accuracy: 0.7968
Epoch 5/50
1905/1905 [==============================] - 22s 12ms/step - loss: 0.1865 - accuracy: 0.8581
Epoch 6/50
1905/1905 [==============================] - 22s 12ms/step - loss: 0.1559 - accuracy: 0.9034
Epoch 7/50
1905/1905 [==============================] - 22s 12ms/step - loss: 0.1202 - accuracy: 0.9255
Epoch 8/50
1905/1905 [==============================] - 22s 11ms/step - loss: 0.1083 - accuracy: 0.9307
Epoch 9/50
1905/1905 [==============================] - 22s 12ms/step - loss: 0.1039 - accuracy: 0.9327
Epoch 10/50
1905/1905 [==============================] - 22s 12m

In [ ]:
# stats for four classes
y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
print("*" * 80)
print("Classification report")
print("*" * 80)
print(classification_report(y_true_labels, y_pred_labels))

********************************************************************************
Classification report
********************************************************************************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1060
           1       0.96      0.93      0.95      1314
           2       0.92      0.97      0.95      1071
           3       0.99      0.98      0.99      1317

    accuracy                           0.97      4762
   macro avg       0.97      0.97      0.97      4762
weighted avg       0.97      0.97      0.97      4762



### Model Training (LSTM)

In [34]:
units = 100
filters = 64
kernel_size = 3
n_features = 6
activation='relu'
n_classes = 5

model_lstm = Sequential()

model_lstm.add(LSTM(units=units, input_shape=(window_size, n_features)))
model_lstm.add(Dense(100, activation='relu'))
model_lstm.add(Dense(n_classes, activation='softmax'))

model_lstm.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               42800     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 505       
Total params: 53,405
Trainable params: 53,405
Non-trainable params: 0
_________________________________________________________________


In [35]:
model_lstm.compile(
    optimizer=optimizers.SGD(lr=0.001),
    loss='binary_crossentropy',
    metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [36]:
model_lstm.fit(X_train, y_train,
        batch_size=10, epochs=50)

Epoch 1/50
1905/1905 [==============================] - 60s 31ms/step - loss: 0.6241 - accuracy: 0.3778
Epoch 2/50
1905/1905 [==============================] - 59s 31ms/step - loss: 0.5372 - accuracy: 0.5093
Epoch 3/50
1905/1905 [==============================] - 59s 31ms/step - loss: 0.4801 - accuracy: 0.5225
Epoch 4/50
1905/1905 [==============================] - 60s 31ms/step - loss: 0.4381 - accuracy: 0.5254
Epoch 5/50
1905/1905 [==============================] - 59s 31ms/step - loss: 0.4038 - accuracy: 0.5370
Epoch 6/50
1905/1905 [==============================] - 59s 31ms/step - loss: 0.3743 - accuracy: 0.5447
Epoch 7/50
1905/1905 [==============================] - 58s 30ms/step - loss: 0.3497 - accuracy: 0.5583
Epoch 8/50
1905/1905 [==============================] - 58s 31ms/step - loss: 0.3296 - accuracy: 0.5718
Epoch 9/50
1905/1905 [==============================] - 58s 31ms/step - loss: 0.3130 - accuracy: 0.5929
Epoch 10/50
1905/1905 [==============================] - 59s 31m

In [38]:
# stats for five classes
y_pred_ohe = model_lstm.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
print("*" * 80)
print("Classification report")
print("*" * 80)
print(classification_report(y_true_labels, y_pred_labels))

********************************************************************************
Classification report
********************************************************************************
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1117
           1       0.97      0.90      0.93      1281
           2       0.91      0.97      0.94      1108
           3       0.79      0.84      0.82       732
           4       0.80      0.61      0.69       524

    accuracy                           0.90      4762
   macro avg       0.88      0.86      0.87      4762
weighted avg       0.90      0.90      0.90      4762



In [39]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model_lstm)
tflite_model = converter.convert()

# Save the model.
with open('model_lstm.tflite', 'wb') as f:
  f.write(tflite_model)

from google.colab import files
files.download('model_lstm.tflite')

INFO:tensorflow:Assets written to: /tmp/tmpo47le2h7/assets


INFO:tensorflow:Assets written to: /tmp/tmpo47le2h7/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
model_lstm.fit(X_train, y_train,
        batch_size=10, epochs=50)

Epoch 1/50
1905/1905 [==============================] - 63s 32ms/step - loss: 0.6251 - accuracy: 0.3823
Epoch 2/50
1905/1905 [==============================] - 60s 32ms/step - loss: 0.5321 - accuracy: 0.5986
Epoch 3/50
1905/1905 [==============================] - 59s 31ms/step - loss: 0.4705 - accuracy: 0.6438
Epoch 4/50
1905/1905 [==============================] - 62s 33ms/step - loss: 0.4207 - accuracy: 0.6506
Epoch 5/50
1905/1905 [==============================] - 59s 31ms/step - loss: 0.3794 - accuracy: 0.6596
Epoch 6/50
1905/1905 [==============================] - 59s 31ms/step - loss: 0.3481 - accuracy: 0.6659
Epoch 7/50
1905/1905 [==============================] - 59s 31ms/step - loss: 0.3246 - accuracy: 0.6753
Epoch 8/50
1905/1905 [==============================] - 58s 30ms/step - loss: 0.3055 - accuracy: 0.7051
Epoch 9/50
1905/1905 [==============================] - 58s 31ms/step - loss: 0.2887 - accuracy: 0.7515
Epoch 10/50
1905/1905 [==============================] - 59s 31m

In [24]:
# stats
y_pred_ohe = model_lstm.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
print("*" * 80)
print("Classification report")
print("*" * 80)
print(classification_report(y_true_labels, y_pred_labels))

********************************************************************************
Classification report
********************************************************************************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1060
           1       0.96      0.93      0.95      1314
           2       0.92      0.97      0.95      1071
           3       0.99      0.98      0.99      1317

    accuracy                           0.97      4762
   macro avg       0.97      0.97      0.97      4762
weighted avg       0.97      0.97      0.97      4762

